# 기획

- 분기별 시가 총액 하위 20% 
- 최초 20일 종가상승률 5% 이상 
- ROA > 10%
- 영업이익률 > 10 %
- 전분기대비 매출성장률 > 10 % 

- 60일 전후 투자수익률   

# 모든 종목 재무제표 크롤링 

In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import os

## 시총데이터에서 2020-2022 종목코드 추출 

## 시총데이터 병합 

In [6]:
marcaplis = os.listdir('./data/marcap/marcap/data/')
# marcapdf = pd.read_csv('./data/marcap/marcap/data/marcap-2022.csv.gz')
marcappath = './data/marcap/marcap/data/'
marcap_lis = []
for i in marcaplis:
    df = pd.read_csv(marcappath+i)
    marcap_lis.append(df)
marcapdf = pd.concat(marcap_lis)

C:\Users\bitcamp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\bitcamp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
marcapdf

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank,Date
0,15760,한국전력공사,KOSPI,NaN,27400.0,2,-400.0,-1.44,27600.0,27800.0,27400.0,118190.0,3.255694e+09,1.687268e+13,615791215,STK,1,1995-05-02
1,5930,삼성전자,KOSPI,NaN,119500.0,2,-1000.0,-0.83,121000.0,121000.0,118500.0,139560.0,1.667674e+10,6.497053e+12,54368645,STK,2,1995-05-02
2,5490,포항종합제철,KOSPI,NaN,65500.0,2,-300.0,-0.46,65800.0,66200.0,65400.0,65970.0,4.334472e+09,6.150569e+12,93901810,STK,3,1995-05-02
3,200,대우중공업,KOSPI,NaN,9800.0,1,300.0,3.16,9500.0,9840.0,9500.0,288380.0,2.805134e+09,3.500639e+12,357208059,STK,4,1995-05-02
4,2610,엘지전자,KOSPI,NaN,31700.0,2,-300.0,-0.94,32000.0,32000.0,31700.0,171460.0,5.452050e+09,2.521027e+12,79527666,STK,5,1995-05-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225121,009275,신원우,KOSPI,NaN,32000.0,2,-450.0,-1.39,31250.0,32450.0,31200.0,326.0,1.024625e+07,2.903680e+09,90740,STK,2619,2022-05-10
225122,215050,비엔디생활건강,KONEX,일반기업부,912.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000e+00,1.934959e+09,2121666,KNX,2620,2022-05-10
225123,329020,오션스톤,KONEX,일반기업부,1700.0,2,-250.0,-12.82,2000.0,2200.0,1660.0,3576.0,6.354905e+06,1.419758e+09,835152,KNX,2621,2022-05-10
225124,267060,명진홀딩스,KONEX,일반기업부,154.0,5,-27.0,-14.92,192.0,192.0,154.0,2158.0,3.327030e+05,1.407838e+09,9141804,KNX,2622,2022-05-10


In [ ]:
# 분기 날짜를 먼저 뽑아야함 
# 2021.3
# 2021.6
# 2021.9
# 2021.12
# 2022.3

In [127]:
# marcapdf.set_index('Date',inplace=True)
marcapdf.head()
marcapdf.tail()

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,Rank
Date,,,,,,,,,,,,,,,,,
2022-05-10,009275,신원우,KOSPI,NaN,32000.0,2,-450.0,-1.39,31250.0,32450.0,31200.0,326.0,10246250.0,2.903680e+09,90740,STK,2619
2022-05-10,215050,비엔디생활건강,KONEX,일반기업부,912.0,0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,1.934959e+09,2121666,KNX,2620
2022-05-10,329020,오션스톤,KONEX,일반기업부,1700.0,2,-250.0,-12.82,2000.0,2200.0,1660.0,3576.0,6354905.0,1.419758e+09,835152,KNX,2621
2022-05-10,267060,명진홀딩스,KONEX,일반기업부,154.0,5,-27.0,-14.92,192.0,192.0,154.0,2158.0,332703.0,1.407838e+09,9141804,KNX,2622
2022-05-10,240340,인터코스,KONEX,일반기업부,64.0,2,-4.0,-5.88,78.0,78.0,64.0,51391.0,3574980.0,2.141647e+08,3346323,KNX,2623


In [155]:
# 재무제표 크롤링을 위한 모든 종목 코드 
codes = marcapdf.loc['2020':'2022'].drop_duplicates('Code')['Code'].tolist()

In [156]:
#2022년 시총을 알 수 있는 모든 종목 
codes

['005930',
 '000660',
 '005935',
 '035420',
 '207940',
 '005380',
 '012330',
 '068270',
 '051910',
 '005490',
 '028260',
 '055550',
 '051900',
 '105560',
 '017670',
 '015760',
 '034730',
 '000270',
 '006400',
 '018260',
 '032830',
 '096770',
 '035720',
 '033780',
 '003550',
 '090430',
 '036570',
 '066570',
 '000810',
 '086790',
 '010950',
 '009150',
 '009540',
 '316140',
 '010130',
 '251270',
 '091990',
 '011170',
 '030200',
 '002790',
 '024110',
 '021240',
 '035250',
 '032640',
 '034220',
 '018880',
 '267250',
 '086280',
 '006800',
 '028300',
 '078930',
 '000720',
 '010140',
 '029780',
 '271560',
 '004020',
 '088980',
 '161390',
 '004990',
 '071050',
 '023530',
 '028050',
 '008770',
 '097950',
 '005830',
 '005940',
 '012750',
 '139480',
 '000120',
 '036460',
 '035760',
 '128940',
 '241560',
 '016360',
 '047810',
 '011070',
 '081660',
 '000210',
 '003670',
 '007070',
 '009830',
 '000100',
 '042660',
 '282330',
 '004170',
 '005387',
 '003410',
 '001040',
 '030000',
 '003490',
 '008930',

## 재무제표 크롤링 

In [157]:
# low20marcap202103 = marcapdf[marcapdf.index == '2021-03-03']
# low20marcap202103
# # low20marcap202103.sort_values(by='Marcap',ascending=True)[:int(len(low20marcap202103)*0.2)]

In [9]:
# # 2022년도 재무제표 
# import requests
# from bs4 import BeautifulSoup

# from selenium.webdriver.common.keys import Keys
# import time
# from selenium import webdriver

# driver  =webdriver.Chrome()
# driver.implicitly_wait(1)
# count = 0

# for code in codes[2200:]:

#     url = 'https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cn=&cmp_cd={}&menuType=block'.\
#             format(code)
#     try:
#         driver.get(url)
# #         element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "cns_Tab22")))        
#         try:
#             driver.find_element_by_id('cns_Tab22').click()

#             html = driver.page_source

#             tables = pd.read_html(html, encoding='utf-8')
#             fin_df = tables[12]

#             fin_df = fin_df.T.iloc[:6]
#             fin_df.columns = fin_df.iloc[0]
#             fin_df.drop('주요재무정보', inplace=True)

#             fin_df.reset_index(inplace=True)
#             fin_df.drop('level_0', axis=1, inplace=True)

#             fin_df.index = fin_df.level_1.apply(lambda x: x.split()[0])
#             fin_df.drop('level_1', axis=1, inplace=True)

#             fin_df = fin_df[['매출액','영업이익','당기순이익','자산총계','부채총계','자본총계','ROE(%)', 'ROA(%)',\
#                     'EPS(원)', 'PER(배)','BPS(원)', 'PBR(배)']]

#             fin_df['code'] = code

#             fin_df.to_csv('./data/네이버재무제표_{}'.format(code))

#             count +=1
#             print(count, end=' ')
#         except:
#             pass
#     except:
#         pass

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_8980/2927220948.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id('cns_Tab22').click()
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\generic.py:4150: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

## 분기별 시총하위20퍼 코드 

In [14]:
marcapdf.set_index('Date',inplace=True)

In [15]:
date_lis = ['2021-03-02','2021-06-01','2021-09-01','2021-12-01','2022-03-02']
def date_marcap_low20(date):
    df = marcapdf[marcapdf.index == date]
    df = df.sort_values(by='Marcap',ascending=True)[:int(len(df)*0.2)]
    return df['Code'].tolist()

In [16]:
# date_marcap_low20()
date_lis_2 = []
for i in date_lis:
    date_lis_2.append(date_marcap_low20(i))

In [17]:
# 분기별 시총 하위 20% 종목들 코드 
low20_marcap_data_codes = date_lis_2
low20_marcap_data_codes[0]

['225860',
 '225850',
 '344860',
 '179720',
 '001529',
 '240340',
 '140660',
 '224020',
 '000325',
 '267810',
 '009275',
 '000547',
 '276240',
 '215050',
 '242350',
 '002787',
 '032685',
 '001527',
 '217910',
 '329020',
 '329050',
 '002785',
 '001525',
 '116100',
 '014915',
 '000227',
 '005965',
 '245450',
 '00781K',
 '114920',
 '308700',
 '001065',
 '016385',
 '021045',
 '001067',
 '238500',
 '270210',
 '000545',
 '090355',
 '185190',
 '093510',
 '183410',
 '311060',
 '014285',
 '158310',
 '266870',
 '347140',
 '266170',
 '004545',
 '208890',
 '323210',
 '000145',
 '080440',
 '058420',
 '011155',
 '006345',
 '335870',
 '288490',
 '114570',
 '270020',
 '121890',
 '310870',
 '310840',
 '005745',
 '373200',
 '321260',
 '140290',
 '323280',
 '271850',
 '349720',
 '084695',
 '003075',
 '311270',
 '26490K',
 '223220',
 '08537M',
 '086460',
 '329560',
 '001755',
 '121060',
 '331520',
 '202960',
 '343510',
 '331380',
 '014825',
 '195440',
 '369370',
 '344050',
 '208850',
 '337450',
 '004985',

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

# driver = webdriver.Chrome('chromedriver.exe')
# driver.get("https://pythondocs.net")

# element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "idname")))

# 재무제표 조건 충족 

## 전체 재무제표 병합

In [18]:
# 전체재무제표 병합
import os 
code_lis = os.listdir('./data/네이버재무제표')

In [19]:
code_lis

['네이버재무제표_000020',
 '네이버재무제표_000040',
 '네이버재무제표_000050',
 '네이버재무제표_000060',
 '네이버재무제표_000070',
 '네이버재무제표_000075',
 '네이버재무제표_000080',
 '네이버재무제표_000087',
 '네이버재무제표_000100',
 '네이버재무제표_000105',
 '네이버재무제표_000120',
 '네이버재무제표_000140',
 '네이버재무제표_000145',
 '네이버재무제표_000150',
 '네이버재무제표_000155',
 '네이버재무제표_000157',
 '네이버재무제표_000180',
 '네이버재무제표_000210',
 '네이버재무제표_000215',
 '네이버재무제표_000220',
 '네이버재무제표_000225',
 '네이버재무제표_000227',
 '네이버재무제표_000230',
 '네이버재무제표_000240',
 '네이버재무제표_000250',
 '네이버재무제표_000270',
 '네이버재무제표_000300',
 '네이버재무제표_000320',
 '네이버재무제표_000325',
 '네이버재무제표_000370',
 '네이버재무제표_000390',
 '네이버재무제표_000400',
 '네이버재무제표_000430',
 '네이버재무제표_000440',
 '네이버재무제표_000480',
 '네이버재무제표_000490',
 '네이버재무제표_000500',
 '네이버재무제표_000520',
 '네이버재무제표_000540',
 '네이버재무제표_000545',
 '네이버재무제표_000547',
 '네이버재무제표_000590',
 '네이버재무제표_000640',
 '네이버재무제표_000650',
 '네이버재무제표_000660',
 '네이버재무제표_000670',
 '네이버재무제표_000680',
 '네이버재무제표_000700',
 '네이버재무제표_000720',
 '네이버재무제표_000725',
 '네이버재무제표_000760',
 '네이버재무제표_000810',
 '네이버재무제표_00

In [20]:
len(code_lis)

2492

In [21]:
path = './data/네이버재무제표/'
df_lis =[]
for i in code_lis:
    df = pd.read_csv(path+i)
    df_lis.append(df)

In [22]:
df_fs = pd.concat(df_lis)

In [23]:
df_fs.head()

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
0,2020/12,687.0,70.0,80.0,4338.0,911.0,3427.0,9.04,7.09,278.0,19.25,11999.0,1.64,20
1,2021/03,718.0,51.0,55.0,4373.0,961.0,3412.0,10.08,7.86,195.0,12.56,11941.0,1.19,20
2,2021/06,758.0,72.0,70.0,4471.0,960.0,3511.0,9.61,7.73,224.0,13.99,12272.0,1.28,20
3,2021/09,691.0,37.0,8.0,4479.0,947.0,3531.0,5.98,4.89,16.0,25.02,12302.0,1.45,20
4,2021/12,763.0,65.0,62.0,4478.0,877.0,3601.0,5.27,4.44,204.0,22.82,12534.0,1.16,20


In [24]:
df_fs['mod_code'] = df_fs['code'].apply(lambda x:str(x).zfill(6))

In [25]:
df_fs.set_index('mod_code',inplace=True)
df_fs

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
000020,2020/12,687.0,70.0,80.0,4338.0,911.0,3427.0,9.04,7.09,278.0,19.25,11999.0,1.64,20
000020,2021/03,718.0,51.0,55.0,4373.0,961.0,3412.0,10.08,7.86,195.0,12.56,11941.0,1.19,20
000020,2021/06,758.0,72.0,70.0,4471.0,960.0,3511.0,9.61,7.73,224.0,13.99,12272.0,1.28,20
000020,2021/09,691.0,37.0,8.0,4479.0,947.0,3531.0,5.98,4.89,16.0,25.02,12302.0,1.45,20
000020,2021/12,763.0,65.0,62.0,4478.0,877.0,3601.0,5.27,4.44,204.0,22.82,12534.0,1.16,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950220,2020/12,NaN,-108.0,-107.0,956.0,114.0,842.0,NaN,NaN,-130.0,NaN,1018.0,0.00,950220
950220,2021/03,NaN,-108.0,-108.0,2040.0,162.0,1879.0,NaN,NaN,-125.0,NaN,1909.0,5.89,950220
950220,2021/06,NaN,-137.0,-137.0,1919.0,155.0,1764.0,-29.88,-27.65,-139.0,NaN,1792.0,5.94,950220


## 2021/03

In [121]:
low20_marcap_data_codes[0]

['225860',
 '225850',
 '344860',
 '179720',
 '001529',
 '240340',
 '140660',
 '224020',
 '000325',
 '267810',
 '009275',
 '000547',
 '276240',
 '215050',
 '242350',
 '002787',
 '032685',
 '001527',
 '217910',
 '329020',
 '329050',
 '002785',
 '001525',
 '116100',
 '014915',
 '000227',
 '005965',
 '245450',
 '00781K',
 '114920',
 '308700',
 '001065',
 '016385',
 '021045',
 '001067',
 '238500',
 '270210',
 '000545',
 '090355',
 '185190',
 '093510',
 '183410',
 '311060',
 '014285',
 '158310',
 '266870',
 '347140',
 '266170',
 '004545',
 '208890',
 '323210',
 '000145',
 '080440',
 '058420',
 '011155',
 '006345',
 '335870',
 '288490',
 '114570',
 '270020',
 '121890',
 '310870',
 '310840',
 '005745',
 '373200',
 '321260',
 '140290',
 '323280',
 '271850',
 '349720',
 '084695',
 '003075',
 '311270',
 '26490K',
 '223220',
 '08537M',
 '086460',
 '329560',
 '001755',
 '121060',
 '331520',
 '202960',
 '343510',
 '331380',
 '014825',
 '195440',
 '369370',
 '344050',
 '208850',
 '337450',
 '004985',

In [122]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[0]:
    low20lis.append(df_fs[df_fs.index == i])

In [123]:
low20lis

[Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
           level_1     매출액  영업이익  당기순이익     자산총계    부채총계    자본총계  ROE(%)  \
 mod_code                                                                  
 001529    2020/12  1669.0 -62.0 -304.0   9968.0  2172.0  7796.0   -2.92   
 001529    2021/03  1560.0 -32.0   42.0   9911.0  2157.0  7754.0   -2.12   
 001529    2021/06  1737.0  43.0   57.0   9975.0  2167.0  7808.0   -2.06   
 001529    2021/09  1721.0  41.0   26.0   9856.0  2022.0  7834.0

In [124]:
df202103 = pd.concat(low20lis)

In [125]:
# df20210302.set_index('level_1',inplace=True)
df202103

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2020/12,1669.0,-62.0,-304.0,9968.0,2172.0,7796.0,-2.92,-2.87,-112.0,NaN,3957.0,0.30,1529
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529
001529,2021/06,1737.0,43.0,57.0,9975.0,2167.0,7808.0,-2.06,-2.23,27.0,NaN,3968.0,0.56,1529
001529,2021/09,1721.0,41.0,26.0,9856.0,2022.0,7834.0,-1.66,-1.79,12.0,NaN,3985.0,0.40,1529
001529,2021/12,1978.0,6.0,-12.0,10072.0,2232.0,7840.0,2.04,1.12,11.0,21.69,4010.0,0.37,1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
002690,2020/12,335.0,-2.0,5.0,1486.0,188.0,1298.0,0.23,0.21,29.0,124.85,7890.0,0.28,2690
002690,2021/03,378.0,13.0,13.0,1563.0,230.0,1333.0,1.40,1.22,76.0,28.14,8105.0,0.38,2690
002690,2021/06,474.0,39.0,33.0,1614.0,238.0,1376.0,3.29,2.83,193.0,17.14,8367.0,0.53,2690


In [126]:
df202103 = df202103[df202103['level_1'] == '2021/03']

In [127]:
df202103

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529
000325,2021/03,2031.0,62.0,61.0,10776.0,5006.0,5770.0,3.68,2.77,145.0,12.78,35615.0,0.36,325
009275,2021/03,1878.0,42.0,44.0,4321.0,2421.0,1900.0,-1.33,-1.04,58.0,NaN,2707.0,0.55,9275
000547,2021/03,12082.0,179.0,149.0,133550.0,126703.0,6848.0,6.35,0.34,228.0,6.10,10508.0,0.39,547
002787,2021/03,796.0,89.0,93.0,2890.0,1609.0,1281.0,24.38,9.04,63.0,18.28,874.0,3.95,2787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290270,2021/03,45.0,1.0,1.0,312.0,48.0,264.0,18.58,15.89,12.0,13.16,2784.0,2.22,290270
000440,2021/03,128.0,3.0,3.0,692.0,95.0,597.0,37.35,30.24,43.0,3.19,13476.0,0.74,440
045510,2021/03,383.0,-2.0,-2.0,781.0,383.0,398.0,1.83,0.98,-6.0,86.35,1246.0,1.54,45510


In [128]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [129]:
df202103 = conmet(df202103)
df202103

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['영업이익률'] = (df['영업이익']/df['매출액'])*100


,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000995,2021/03,2437.0,606.0,478.0,11962.0,3678.0,8285.0,20.98,13.55,1074.0,15.78,19062.0,2.95,995,100.0,24.866639


In [130]:
df202103_conmetcode = df202103.index.tolist()
df202103_conmetcode

['000995']

## 2021/06

In [132]:
low20_marcap_data_codes[1]

['225860',
 '225850',
 '267810',
 '242350',
 '093510',
 '240340',
 '179720',
 '008800',
 '140660',
 '215050',
 '245450',
 '276240',
 '116100',
 '002787',
 '195440',
 '114920',
 '217910',
 '288490',
 '308700',
 '000547',
 '009275',
 '032685',
 '270210',
 '073070',
 '001529',
 '329020',
 '158310',
 '217320',
 '002785',
 '014915',
 '270020',
 '080440',
 '140290',
 '021045',
 '001067',
 '114570',
 '121890',
 '335870',
 '323210',
 '347140',
 '00781K',
 '001527',
 '016385',
 '310840',
 '001525',
 '058420',
 '373200',
 '349720',
 '321260',
 '014285',
 '311270',
 '208890',
 '285770',
 '271850',
 '331380',
 '343510',
 '344860',
 '001065',
 '011155',
 '000545',
 '08537M',
 '311060',
 '310870',
 '369370',
 '330990',
 '329560',
 '000227',
 '033600',
 '003075',
 '331520',
 '332710',
 '121060',
 '351320',
 '353070',
 '006345',
 '365590',
 '086460',
 '266870',
 '340440',
 '313750',
 '014825',
 '000145',
 '341160',
 '002995',
 '307870',
 '323280',
 '038160',
 '307160',
 '058220',
 '004565',
 '373340',

In [133]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[1]:
    low20lis.append(df_fs[df_fs.index == i])

In [134]:
low20lis

[Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: 

In [135]:
df202106 = pd.concat(low20lis)

In [136]:
# df20210302.set_index('level_1',inplace=True)
df202106

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
002787,2020/12,1095.0,49.0,121.0,2829.0,1641.0,1188.0,18.86,6.70,82.0,18.65,811.0,3.20,2787
002787,2021/03,796.0,89.0,93.0,2890.0,1609.0,1281.0,24.38,9.04,63.0,18.28,874.0,3.95,2787
002787,2021/06,1063.0,167.0,226.0,3150.0,1643.0,1507.0,36.86,14.77,154.0,8.50,1028.0,2.64,2787
002787,2021/09,1042.0,87.0,83.0,3095.0,1506.0,1590.0,39.48,16.92,57.0,6.48,1085.0,2.13,2787
002787,2021/12,1542.0,90.0,132.0,3346.0,1637.0,1710.0,36.86,17.30,90.0,5.98,1166.0,1.87,2787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
010400,2020/12,363.0,58.0,49.0,1210.0,196.0,1014.0,1.69,1.39,640.0,34.87,15094.0,0.53,10400
010400,2021/03,163.0,-47.0,-40.0,1173.0,205.0,968.0,-0.80,-0.64,-525.0,NaN,14407.0,0.55,10400
010400,2021/06,247.0,-7.0,0.0,1168.0,203.0,965.0,0.82,0.67,-5.0,84.59,14357.0,0.62,10400


In [137]:
df202106 = df202106[df202106['level_1'] == '2021/06']

In [138]:
df202106

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
002787,2021/06,1063.0,167.0,226.0,3150.0,1643.0,1507.0,36.86,14.77,154.0,8.50,1028.0,2.64,2787
000547,2021/06,11073.0,122.0,128.0,134522.0,127487.0,7035.0,5.10,0.28,196.0,8.40,10795.0,0.44,547
009275,2021/06,2107.0,51.0,3.0,4465.0,2564.0,1901.0,1.60,0.23,10.0,53.69,2732.0,0.71,9275
032685,2021/06,208.0,32.0,29.0,673.0,180.0,494.0,17.00,13.71,67.0,13.12,1195.0,1.93,32685
001529,2021/06,1737.0,43.0,57.0,9975.0,2167.0,7808.0,-2.06,-2.23,27.0,NaN,3968.0,0.56,1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
091340,2021/06,797.0,36.0,22.0,2700.0,1459.0,1240.0,-3.99,-1.74,121.0,NaN,8219.0,0.71,91340
096640,2021/06,54.0,-4.0,-37.0,559.0,286.0,273.0,-56.99,-24.55,-108.0,NaN,808.0,2.68,96640
313760,2021/06,61.0,-13.0,-9.0,447.0,113.0,335.0,-9.87,-7.12,-180.0,NaN,6881.0,1.98,313760


In [139]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [140]:
df202106 = conmet(df202106)
df202106

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['영업이익률'] = (df['영업이익']/df['매출액'])*100


,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
002785,2021/06,1063.0,167.0,226.0,3150.0,1643.0,1507.0,36.86,14.77,154.0,8.50,1028.0,2.64,2785,99.623706,15.710254
096870,2021/06,33.0,4.0,5.0,143.0,19.0,124.0,23.30,20.07,70.0,14.16,1851.0,2.95,96870,48.484848,12.121212
189860,2021/06,190.0,26.0,28.0,551.0,137.0,414.0,18.27,13.03,291.0,9.02,4270.0,1.52,189860,67.368421,13.684211


In [141]:
df202106_conmetcode = df202106.index.tolist()
df202106_conmetcode

['002785', '096870', '189860']

## 2021/09

In [142]:
low20_marcap_data_codes[2]

['225860',
 '225850',
 '267810',
 '245450',
 '140660',
 '242350',
 '240340',
 '093510',
 '215050',
 '001529',
 '270210',
 '116100',
 '000547',
 '009275',
 '114920',
 '002787',
 '179720',
 '032685',
 '308700',
 '288490',
 '276240',
 '267060',
 '001527',
 '001525',
 '318660',
 '329020',
 '217910',
 '002785',
 '140290',
 '001067',
 '014915',
 '016385',
 '217320',
 '270020',
 '021045',
 '080440',
 '310840',
 '001065',
 '114570',
 '00781K',
 '121890',
 '323210',
 '347140',
 '000545',
 '058420',
 '011155',
 '285770',
 '000227',
 '323280',
 '349720',
 '185190',
 '373200',
 '000145',
 '329560',
 '014285',
 '331520',
 '033430',
 '121060',
 '004565',
 '335870',
 '343510',
 '004545',
 '08537M',
 '266870',
 '353070',
 '005745',
 '330990',
 '311060',
 '271850',
 '002995',
 '369370',
 '344050',
 '332710',
 '366330',
 '373340',
 '014825',
 '003075',
 '000325',
 '372290',
 '313750',
 '337450',
 '367360',
 '341160',
 '006345',
 '058220',
 '189350',
 '004415',
 '365590',
 '086460',
 '322190',
 '353060',

In [143]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[2]:
    low20lis.append(df_fs[df_fs.index == i])

In [144]:
low20lis

[Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: 

In [145]:
df202109 = pd.concat(low20lis)

In [146]:
# df20210302.set_index('level_1',inplace=True)
df202109

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2020/12,1669.0,-62.0,-304.0,9968.0,2172.0,7796.0,-2.92,-2.87,-112.0,NaN,3957.0,0.30,1529
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529
001529,2021/06,1737.0,43.0,57.0,9975.0,2167.0,7808.0,-2.06,-2.23,27.0,NaN,3968.0,0.56,1529
001529,2021/09,1721.0,41.0,26.0,9856.0,2022.0,7834.0,-1.66,-1.79,12.0,NaN,3985.0,0.40,1529
001529,2021/12,1978.0,6.0,-12.0,10072.0,2232.0,7840.0,2.04,1.12,11.0,21.69,4010.0,0.37,1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212560,2020/12,341.0,15.0,9.0,1444.0,617.0,827.0,4.37,2.48,111.0,11.18,10503.0,0.48,212560
212560,2021/03,361.0,17.0,18.0,1421.0,579.0,841.0,5.61,3.25,228.0,14.64,10681.0,0.80,212560
212560,2021/06,366.0,22.0,31.0,1404.0,532.0,872.0,8.86,5.35,393.0,11.53,11073.0,0.98,212560


In [147]:
df202109 = df202109[df202109['level_1'] == '2021/09']

In [148]:
df202109

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2021/09,1721.0,41.0,26.0,9856.0,2022.0,7834.0,-1.66,-1.79,12.0,NaN,3985.0,0.40,1529
000547,2021/09,11329.0,419.0,322.0,136868.0,129689.0,7179.0,9.14,0.51,494.0,3.61,11017.0,0.34,547
009275,2021/09,2383.0,61.0,21.0,4932.0,2894.0,2039.0,1.63,0.19,30.0,81.70,2716.0,1.17,9275
002787,2021/09,1042.0,87.0,83.0,3095.0,1506.0,1590.0,39.48,16.92,57.0,6.48,1085.0,2.13,2787
032685,2021/09,300.0,82.0,71.0,829.0,252.0,577.0,31.76,22.96,177.0,5.90,1401.0,1.62,32685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
012620,2021/09,794.0,42.0,128.0,2828.0,1441.0,1387.0,14.46,7.17,2903.0,3.37,31523.0,0.46,12620
050120,2021/09,324.0,24.0,21.0,9585.0,8773.0,813.0,0.51,0.05,154.0,147.98,6060.0,0.74,50120
091340,2021/09,824.0,0.0,22.0,3046.0,1767.0,1279.0,-0.93,-0.56,143.0,NaN,8447.0,0.65,91340


In [149]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [150]:
df202109 = conmet(df202109)
df202109

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['영업이익률'] = (df['영업이익']/df['매출액'])*100


,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000995,2021/09,3284.0,1190.0,927.0,13625.0,3819.0,9806.0,24.18,16.91,2082.0,11.41,22563.0,2.42,995,100.000000,36.236297
36328K,2021/09,780.0,117.0,769.0,21021.0,5781.0,15240.0,45.78,23.57,1669.0,2.47,35153.0,0.82,36328K,68.333333,15.000000
096870,2021/09,25.0,4.0,6.0,149.0,20.0,129.0,22.66,19.36,87.0,11.62,1938.0,2.37,96870,68.000000,16.000000
065420,2021/09,86.0,12.0,14.0,192.0,21.0,171.0,23.86,20.03,20.0,12.34,252.0,2.54,65420,69.767442,13.953488
072130,2021/09,89.0,14.0,22.0,608.0,112.0,496.0,38.97,30.59,170.0,3.93,4293.0,1.13,72130,37.078652,15.730337
900340,2021/09,387.0,92.0,80.0,1836.0,311.0,1525.0,16.08,13.39,166.0,2.98,3180.0,0.43,900340,86.046512,23.772610


In [151]:
df202109_conmetcode = df202109.index.tolist()
df202109_conmetcode

['000995', '36328K', '096870', '065420', '072130', '900340']

## 2021/12

In [152]:
low20_marcap_data_codes[3]

['225860',
 '225850',
 '329020',
 '267810',
 '001529',
 '267060',
 '245450',
 '114920',
 '240340',
 '009275',
 '140660',
 '215050',
 '318660',
 '000547',
 '032685',
 '270210',
 '276240',
 '002787',
 '001525',
 '001527',
 '002785',
 '016385',
 '116100',
 '000227',
 '014915',
 '001067',
 '000545',
 '308700',
 '021045',
 '288490',
 '001065',
 '285770',
 '347140',
 '008355',
 '270020',
 '000325',
 '323210',
 '373340',
 '335870',
 '005965',
 '000145',
 '140290',
 '217910',
 '014285',
 '114570',
 '121890',
 '323280',
 '005745',
 '006345',
 '388790',
 '00781K',
 '004105',
 '011155',
 '349720',
 '004545',
 '08537M',
 '185190',
 '058420',
 '004565',
 '329560',
 '331520',
 '002995',
 '004415',
 '179720',
 '373200',
 '003075',
 '343510',
 '014825',
 '090355',
 '266870',
 '208890',
 '348840',
 '317320',
 '337450',
 '330990',
 '353070',
 '344050',
 '001745',
 '372290',
 '084695',
 '332710',
 '217320',
 '012205',
 '327970',
 '26490K',
 '366330',
 '313750',
 '242350',
 '004985',
 '086460',
 '271850',

In [153]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[3]:
    low20lis.append(df_fs[df_fs.index == i])

In [154]:
low20lis

[Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
           level_1     매출액  영업이익  당기순이익     자산총계    부채총계    자본총계  ROE(%)  \
 mod_code                                                                  
 001529    2020/12  1669.0 -62.0 -304.0   9968.0  2172.0  7796.0   -2.92   
 001529    2021/03  1560.0 -32.0   42.0   9911.0  2157.0  7754.0   -2.12   
 001529    2021/06  1737.0  43.0   57.0   9975.0  2167.0  7808.0   -2.06   
 001529    2021/09  1721.0  41.0   26.0   9856.0  2022.0  7834.0

In [155]:
df202112 = pd.concat(low20lis)

In [156]:
# df20210302.set_index('level_1',inplace=True)
df202112

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2020/12,1669.0,-62.0,-304.0,9968.0,2172.0,7796.0,-2.92,-2.87,-112.0,NaN,3957.0,0.30,1529
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529
001529,2021/06,1737.0,43.0,57.0,9975.0,2167.0,7808.0,-2.06,-2.23,27.0,NaN,3968.0,0.56,1529
001529,2021/09,1721.0,41.0,26.0,9856.0,2022.0,7834.0,-1.66,-1.79,12.0,NaN,3985.0,0.40,1529
001529,2021/12,1978.0,6.0,-12.0,10072.0,2232.0,7840.0,2.04,1.12,11.0,21.69,4010.0,0.37,1529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238120,2020/12,25.0,1.0,7.0,255.0,40.0,215.0,2.21,1.87,98.0,152.01,2985.0,3.32,238120
238120,2021/03,22.0,1.0,16.0,293.0,63.0,230.0,8.40,6.83,219.0,31.89,3198.0,2.57,238120
238120,2021/06,31.0,9.0,11.0,295.0,55.0,239.0,14.92,12.38,154.0,19.78,3330.0,2.76,238120


In [157]:
df202112 = df202112[df202112['level_1'] == '2021/12']

In [158]:
df202112

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2021/12,1978.0,6.0,-12.0,10072.0,2232.0,7840.0,2.04,1.12,11.0,21.69,4010.0,0.37,1529
009275,2021/12,2426.0,63.0,37.0,5036.0,2921.0,2115.0,6.01,2.31,40.0,21.10,2665.0,1.10,9275
000547,2021/12,10663.0,52.0,22.0,137623.0,130716.0,6906.0,8.77,0.46,34.0,3.78,10599.0,0.34,547
032685,2021/12,247.0,58.0,27.0,1017.0,374.0,643.0,32.32,18.35,92.0,5.85,1522.0,1.61,32685
002787,2021/12,1542.0,90.0,132.0,3346.0,1637.0,1710.0,36.86,17.30,90.0,5.98,1166.0,1.87,2787
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
050120,2021/12,336.0,-114.0,-54.0,10827.0,9669.0,1159.0,11.62,0.60,-134.0,7.50,6347.0,0.78,50120
094970,2021/12,224.0,69.0,50.0,1251.0,498.0,753.0,22.81,12.38,300.0,4.67,4505.0,0.94,94970
002820,2021/12,1297.0,-226.0,-365.0,5662.0,3979.0,1683.0,-17.83,-6.45,-2765.0,NaN,14375.0,0.35,2820


In [159]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [160]:
df202112 = conmet(df202112)
df202112

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['영업이익률'] = (df['영업이익']/df['매출액'])*100


,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000995,2021/12,3679.0,1381.0,1174.0,15499.0,4439.0,11060.0,33.35,23.23,2637.0,10.21,25449.0,2.86,995,100.000000,37.537374
011300,2021/12,170.0,79.0,294.0,1416.0,1054.0,362.0,88.21,10.62,479.0,2.05,636.0,1.24,11300,79.411765,46.470588
238120,2021/12,40.0,11.0,23.0,345.0,75.0,270.0,22.87,18.49,316.0,11.12,3754.0,2.28,238120,87.500000,27.500000


In [161]:
df202112_conmetcode = df202112.index.tolist()
df202112_conmetcode

['000995', '011300', '238120']

## 2022/03

In [162]:
low20_marcap_data_codes[4]

['225860',
 '225850',
 '329020',
 '267060',
 '215050',
 '308700',
 '267810',
 '245450',
 '240340',
 '009275',
 '318660',
 '001529',
 '140660',
 '114920',
 '270210',
 '276240',
 '050540',
 '000547',
 '032685',
 '285770',
 '217910',
 '001525',
 '001527',
 '002787',
 '208890',
 '116100',
 '199290',
 '014915',
 '002785',
 '016385',
 '140290',
 '288490',
 '405350',
 '021045',
 '000545',
 '347140',
 '270020',
 '179720',
 '001065',
 '001067',
 '373340',
 '335870',
 '000227',
 '400560',
 '008355',
 '121890',
 '005965',
 '266870',
 '388790',
 '323280',
 '004105',
 '349720',
 '004565',
 '271850',
 '00781K',
 '011155',
 '001745',
 '058420',
 '329560',
 '005745',
 '000145',
 '006345',
 '014285',
 '004545',
 '331520',
 '002995',
 '343510',
 '078650',
 '014825',
 '185190',
 '330990',
 '337450',
 '004415',
 '353070',
 '238500',
 '344050',
 '000325',
 '327970',
 '003075',
 '258050',
 '372290',
 '332710',
 '090355',
 '397880',
 '121060',
 '217320',
 '012205',
 '386580',
 '001755',
 '365590',
 '366330',

In [163]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[4]:
    low20lis.append(df_fs[df_fs.index == i])

In [164]:
low20lis

[Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: [level_1, 매출액, 영업이익, 당기순이익, 자산총계, 부채총계, 자본총계, ROE(%), ROA(%), EPS(원), PER(배), BPS(원), PBR(배), code]
 Index: [],
 Empty DataFrame
 Columns: 

In [165]:
df202203 = pd.concat(low20lis)

In [166]:
# df20210302.set_index('level_1',inplace=True)
df202203

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
009275,2020/12,1564.0,2.0,-59.0,4058.0,2202.0,1856.0,-3.91,-2.28,-59.0,NaN,2611.0,0.54,9275
009275,2021/03,1878.0,42.0,44.0,4321.0,2421.0,1900.0,-1.33,-1.04,58.0,NaN,2707.0,0.55,9275
009275,2021/06,2107.0,51.0,3.0,4465.0,2564.0,1901.0,1.60,0.23,10.0,53.69,2732.0,0.71,9275
009275,2021/09,2383.0,61.0,21.0,4932.0,2894.0,2039.0,1.63,0.19,30.0,81.70,2716.0,1.17,9275
009275,2021/12,2426.0,63.0,37.0,5036.0,2921.0,2115.0,6.01,2.31,40.0,21.10,2665.0,1.10,9275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
046310,2020/12,361.0,-6.0,-47.0,964.0,430.0,534.0,-13.73,-0.80,-319.0,NaN,2266.0,1.34,46310
046310,2021/03,418.0,44.0,45.0,1022.0,439.0,582.0,-6.86,0.04,161.0,NaN,2448.0,1.32,46310
046310,2021/06,330.0,19.0,13.0,1021.0,425.0,596.0,-2.48,1.86,60.0,NaN,2515.0,1.93,46310


In [167]:
df202203 = df202203[df202203['level_1'] == '2022/03']

In [168]:
df202203

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
347140,2022/03,0.0,0.0,0.0,59.0,4.0,54.0,-0.11,-0.10,0.0,NaN,1809.0,1.17,347140
373340,2022/03,0.0,0.0,0.0,68.0,11.0,58.0,NaN,NaN,-4.0,NaN,1911.0,1.20,373340
323280,2022/03,0.0,0.0,0.0,80.0,11.0,69.0,0.20,0.17,-2.0,543.25,1974.0,1.07,323280
344050,2022/03,0.0,0.0,0.0,99.0,14.0,85.0,-0.11,-0.10,-5.0,NaN,1970.0,1.11,344050
367360,2022/03,0.0,0.0,0.0,98.0,11.0,88.0,0.12,0.10,0.0,1017.28,1809.0,1.18,367360
388800,2022/03,0.0,0.0,0.0,98.0,9.0,89.0,NaN,NaN,-2.0,NaN,1783.0,1.20,388800
367340,2022/03,0.0,0.0,0.0,119.0,9.0,110.0,0.44,0.41,3.0,267.47,1813.0,1.18,367340
328380,2022/03,0.0,-1.0,0.0,149.0,23.0,126.0,-0.43,-0.36,-7.0,NaN,1940.0,1.06,328380
336570,2022/03,0.0,0.0,0.0,114.0,8.0,106.0,-0.33,-0.31,-2.0,NaN,1892.0,1.41,336570


In [169]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [170]:
df202203 = conmet(df202203)
df202203

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3111229385.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['영업이익률'] = (df['영업이익']/df['매출액'])*100


,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
115610,2022/03,53.0,6.0,8.0,130.0,34.0,95.0,16.88,12.51,52.0,23.42,623.0,3.57,115610,100.000000,11.320755
032580,2022/03,166.0,27.0,26.0,531.0,99.0,432.0,15.32,11.53,77.0,9.82,1304.0,1.40,32580,65.662651,16.265060


In [106]:
df202203_conmetcode = df202203.index.tolist()
df202203_conmetcode

['115610', '032580']

## 조건 충족하는 데이터 프레임 병합 

In [171]:
# conmet_lis = ['df202103','df202106','df202109','df202112','df202203']
# conmet_lis2 = []
# for i in conmet_lis:
#     conmet_lis2.append(i)

# 'df202103','df202106','df202109','df202112','df202203'
conmet_lis = []

In [172]:
conmet_lis.append(df202103)    
conmet_lis.append(df202106)    
conmet_lis.append(df202109)    
conmet_lis.append(df202112)    
conmet_lis.append(df202203)    

In [173]:
conmet_lis

[          level_1     매출액   영업이익  당기순이익     자산총계    부채총계    자본총계  ROE(%)  \
 mod_code                                                                   
 000995    2021/03  2437.0  606.0  478.0  11962.0  3678.0  8285.0   20.98   
 
           ROA(%)  EPS(원)  PER(배)   BPS(원)  PBR(배) code  전분기대비매출성장률      영업이익률  
 mod_code                                                                       
 000995     13.55  1074.0   15.78  19062.0    2.95  995       100.0  24.866639  ,
           level_1     매출액   영업이익  당기순이익    자산총계    부채총계    자본총계  ROE(%)  \
 mod_code                                                                  
 002785    2021/06  1063.0  167.0  226.0  3150.0  1643.0  1507.0   36.86   
 096870    2021/06    33.0    4.0    5.0   143.0    19.0   124.0   23.30   
 189860    2021/06   190.0   26.0   28.0   551.0   137.0   414.0   18.27   
 
           ROA(%)  EPS(원)  PER(배)  BPS(원)  PBR(배)    code  전분기대비매출성장률  \
 mod_code                                                           

In [174]:
df_conmet = pd.concat(conmet_lis)

In [175]:
df_conmet

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000995,2021/03,2437.0,606.0,478.0,11962.0,3678.0,8285.0,20.98,13.55,1074.0,15.78,19062.0,2.95,995,100.000000,24.866639
002785,2021/06,1063.0,167.0,226.0,3150.0,1643.0,1507.0,36.86,14.77,154.0,8.50,1028.0,2.64,2785,99.623706,15.710254
096870,2021/06,33.0,4.0,5.0,143.0,19.0,124.0,23.30,20.07,70.0,14.16,1851.0,2.95,96870,48.484848,12.121212
189860,2021/06,190.0,26.0,28.0,551.0,137.0,414.0,18.27,13.03,291.0,9.02,4270.0,1.52,189860,67.368421,13.684211
000995,2021/09,3284.0,1190.0,927.0,13625.0,3819.0,9806.0,24.18,16.91,2082.0,11.41,22563.0,2.42,995,100.000000,36.236297
36328K,2021/09,780.0,117.0,769.0,21021.0,5781.0,15240.0,45.78,23.57,1669.0,2.47,35153.0,0.82,36328K,68.333333,15.000000
096870,2021/09,25.0,4.0,6.0,149.0,20.0,129.0,22.66,19.36,87.0,11.62,1938.0,2.37,96870,68.000000,16.000000
065420,2021/09,86.0,12.0,14.0,192.0,21.0,171.0,23.86,20.03,20.0,12.34,252.0,2.54,65420,69.767442,13.953488
072130,2021/09,89.0,14.0,22.0,608.0,112.0,496.0,38.97,30.59,170.0,3.93,4293.0,1.13,72130,37.078652,15.730337


In [181]:
concodes = df_conmet.index.drop_duplicates().tolist()
concodes

['000995',
 '002785',
 '096870',
 '189860',
 '36328K',
 '065420',
 '072130',
 '900340',
 '011300',
 '238120',
 '115610',
 '032580']

# 수익률 조건 충족 

In [182]:
import FinanceDataReader as fdr

In [279]:
def rtn20_60(df):
    df['rtn20'] = df['Close'].pct_change(20)
    df['rtn60'] = df['Close'].pct_change(60)
    df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
    df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
    df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
    
    df = df[df['rtn20'] > 0.05]
    return df

In [280]:
# 데이터프레임을 만들어서 인덱스만 뽑아내고 콘캣 
conmet_rtn20 =[]
for i in concodes:
    df = fdr.DataReader(i,'2021')
    df['code'] = df['Close'].apply(lambda x:i)    
    conmet_rtn20.append(rtn20_60(df))

In [281]:
conmet_rtn20

[              Open    High     Low   Close  Volume    Change    code  \
 Date                                                                   
 2021-04-01  103000  104500  101500  103500     503  0.004854  000995   
 2021-04-02  105500  105500  102000  102500     574 -0.009662  000995   
 2021-04-05  104000  109000  103000  108000    1065  0.053659  000995   
 2021-04-06  110000  114500  108500  114500    1746  0.060185  000995   
 2021-04-07  114000  114000  111500  113500     676 -0.008734  000995   
 2021-04-08  111500  113500  110500  110500     702 -0.026432  000995   
 2021-04-09  112500  112500  110000  110000     484 -0.004525  000995   
 2021-04-12  110500  114000  110000  114000     341  0.036364  000995   
 2021-04-13  115500  115500  114000  115000    1139  0.008772  000995   
 2021-04-14  117000  117000  115000  116000     532  0.008696  000995   
 2021-04-15  117500  117500  115000  115500     537 -0.004310  000995   
 2021-04-16  115000  115000  108500  113000     727

In [282]:
rtnconmetdf = pd.concat(conmet_rtn20)
rtnconmetdf

,Open,High,Low,Close,Volume,Change,code,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,,
2021-04-01,103000,104500,101500,103500,503,0.004854,000995,0.068111,-0.100000,0.198068,0.149758,0.169082
2021-04-02,105500,105500,102000,102500,574,-0.009662,000995,0.075551,-0.096916,0.180488,0.200000,0.156098
2021-04-05,104000,109000,103000,108000,1065,0.053659,000995,0.155080,-0.129032,0.125000,0.157407,0.092593
2021-04-06,110000,114500,108500,114500,1746,0.060185,000995,0.254107,-0.053719,0.061135,0.091703,0.013100
2021-04-07,114000,114000,111500,113500,676,-0.008734,000995,0.233696,-0.061983,0.061674,0.114537,0.074890
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19,1905,1930,1845,1865,891078,-0.020997,032580,0.113433,0.180380,NaN,NaN,NaN
2022-04-20,1885,1900,1860,1890,678852,0.013405,032580,0.148936,0.181250,NaN,NaN,NaN
2022-04-21,1945,1995,1885,1905,2959402,0.007937,032580,0.144144,0.179567,NaN,NaN,NaN


In [283]:
df202106_conmetcode

['002785', '096870', '189860']

In [284]:
temp_lis = []
temp_df = rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[0]].loc['2021-06-01']

In [285]:
temp_lis.append(pd.DataFrame(temp_df).T)
pd.concat(temp_lis)

,Open,High,Low,Close,Volume,Change,code,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
2021-06-01,7080,7250,6900,7240,26288,0.034286,002785,0.080597,0.458207,-0.129834,-0.006906,0.052486


In [286]:
rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[2]].loc['2021-06-01']

KeyError: '2021-06-01'

In [287]:
temp_lis = []

In [288]:
for i in range(len(df202103_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202103_conmetcode[i]].loc['2021-03-02']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [289]:
temp_lis

[]

In [290]:
for i in range(len(df202106_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[i]].loc['2021-06-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [291]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [292]:
for i in range(len(df202109_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202109_conmetcode[i]].loc['2021-09-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [293]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [294]:
for i in range(len(df202112_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202112_conmetcode[i]].loc['2021-12-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [295]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [296]:
for i in range(len(df202203_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202203_conmetcode[i]].loc['2022-03-02']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [297]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ,
             Open  High   Low Close  Volume    Change    code     rtn20  \
 2022-03-02  1690  1710  1640  1705  835145  0.008876  032580  0.249084   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2022-03-02  0.121711       NaN -0.002933  0.067449  ]

# 수익률 조건 충족 하지 않음 
- PER < 10 or not 
- PBR < 1
- 수익률 5% 빼기 
- 수익률 < 10 %

In [ ]:
df_fs

In [424]:
def conmet(df):
    df['전분기대비매출성장률'] = ((df['매출액'] - df['매출액'].shift())/df['매출액'])*100
    df['영업이익률'] = (df['영업이익']/df['매출액'])*100
    df = df[(df['ROA(%)'] > 10) & (df['영업이익률'] > 10) & (df['전분기대비매출성장률'] > 10)]
    return df 

In [420]:
df_conmetf = conmet(df_fs) #재무제표 조건충족 
df_conmetf

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000970,2021/06,1523.0,213.0,177.0,3888.0,810.0,3078.0,16.54,12.60,777.0,4.85,13942.0,0.72,970,31.779383,13.985555
001880,2021/06,4939.0,539.0,363.0,15394.0,6652.0,8742.0,21.17,12.01,1558.0,5.56,37514.0,0.95,1880,15.468718,10.913140
002170,2021/06,512.0,106.0,167.0,4537.0,536.0,4002.0,16.42,14.63,5577.0,3.86,140435.0,0.56,2170,13.085938,20.703125
003650,2021/12,1129.0,113.0,96.0,3476.0,456.0,3020.0,11.45,10.05,5513.0,4.15,199594.0,0.40,3650,14.703277,10.008857
003800,2021/06,911.0,249.0,203.0,6048.0,724.0,5324.0,11.88,10.58,1830.0,8.67,51930.0,0.91,3800,11.196487,27.332602
004000,2021/03,3439.0,355.0,405.0,19172.0,2652.0,16520.0,12.48,10.38,1570.0,7.72,64861.0,0.91,4000,11.689445,10.322768
004000,2021/06,4217.0,539.0,1140.0,20717.0,3057.0,17661.0,14.76,12.73,4419.0,6.94,69339.0,0.95,4000,18.449134,12.781598
004590,2021/12,285.0,44.0,130.0,1725.0,398.0,1327.0,17.79,13.34,863.0,4.42,8612.0,0.75,4590,23.508772,15.438596
004800,2021/12,10679.0,1466.0,900.0,52579.0,22656.0,29923.0,18.04,11.16,3660.0,4.55,131860.0,0.72,4800,19.346381,13.727877


In [425]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
low20lis = []
for i in low20_marcap_data_codes[0]:
    low20lis.append(df_fs[df_fs.index == i])
tyu = pd.concat(low20lis)
tyu

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
001529,2020/12,1669.0,-62.0,-304.0,9968.0,2172.0,7796.0,-2.92,-2.87,-112.0,NaN,3957.0,0.30,1529,-18.514080,-3.714799
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529,-6.987179,-2.051282
001529,2021/06,1737.0,43.0,57.0,9975.0,2167.0,7808.0,-2.06,-2.23,27.0,NaN,3968.0,0.56,1529,10.189983,2.475533
001529,2021/09,1721.0,41.0,26.0,9856.0,2022.0,7834.0,-1.66,-1.79,12.0,NaN,3985.0,0.40,1529,-0.929692,2.382336
001529,2021/12,1978.0,6.0,-12.0,10072.0,2232.0,7840.0,2.04,1.12,11.0,21.69,4010.0,0.37,1529,12.992922,0.303337
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
002690,2020/12,335.0,-2.0,5.0,1486.0,188.0,1298.0,0.23,0.21,29.0,124.85,7890.0,0.28,2690,36.716418,-0.597015
002690,2021/03,378.0,13.0,13.0,1563.0,230.0,1333.0,1.40,1.22,76.0,28.14,8105.0,0.38,2690,11.375661,3.439153
002690,2021/06,474.0,39.0,33.0,1614.0,238.0,1376.0,3.29,2.83,193.0,17.14,8367.0,0.53,2690,20.253165,8.227848


In [438]:
asd_code = conmet(tyu)[conmet(tyu)['level_1'] == '2021/06'].index.tolist()

In [448]:
# df = fdr.DataReader('023790','2021')
# df[df.index == '2021-06-01']
lis = []
for i in asd_code:
    df = fdr.DataReader(i,'2021')
    df['rtn20'] = df['Close'].pct_change(20)
    df['rtn60'] = df['Close'].pct_change(60)
    df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
    df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
    df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
    lis.append(df[df.index == '2021-06-01'])
asd = pd.concat(lis)

In [454]:
asd

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-06-01,14350,15200,14250,15150,26177,0.044828,0.134831,0.317391,0.042904,0.075908,0.151815
2021-06-01,37100,38900,37100,37300,2192,0.027548,0.165625,0.407547,-0.158177,0.018767,0.018767
2021-06-01,7080,7250,6900,7240,26288,0.034286,0.080597,0.458207,-0.129834,-0.006906,0.052486
2021-06-01,120000,123500,120000,123000,463,0.033613,0.037975,0.290661,-0.085366,-0.065041,-0.016260
2021-06-01,5030,5030,4840,4840,816235,-0.023209,0.449102,0.979550,-0.185950,-0.138430,-0.063017
2021-06-01,3551,3551,3416,3476,146410,0.005787,-0.079693,0.334357,0.056387,0.047756,0.066456
2021-06-01,4200,4315,4200,4270,102406,0.016667,-0.099156,0.113429,1.459016,1.599532,0.503513
2021-06-01,4170,4315,4165,4215,154936,0.013221,0.019347,0.028049,0.036773,0.100830,0.255042
2021-06-01,7600,7900,7540,7720,391226,0.030708,0.138643,0.421731,-0.106218,0.001295,0.012953


## 다시한번 확인 어쨌든 수익률 

In [451]:
asd['rtn60fore'].mean()*100

10.41392505388382

In [452]:
asd['rtn40fore'].mean()*100

17.858365084864527

In [453]:
asd['rtn20fore'].mean()*100

10.07481966246566

In [447]:
df['rtn20'] = df['Close'].pct_change(20)
df['rtn60'] = df['Close'].pct_change(60)
df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-06-01,14350,15200,14250,15150,26177,0.044828,NaN,NaN,NaN,NaN,NaN
2021-06-01,37100,38900,37100,37300,2192,0.027548,NaN,NaN,NaN,NaN,NaN
2021-06-01,7080,7250,6900,7240,26288,0.034286,NaN,NaN,NaN,NaN,NaN
2021-06-01,120000,123500,120000,123000,463,0.033613,NaN,NaN,NaN,NaN,NaN
2021-06-01,5030,5030,4840,4840,816235,-0.023209,NaN,NaN,NaN,NaN,NaN
2021-06-01,3551,3551,3416,3476,146410,0.005787,NaN,NaN,NaN,NaN,NaN
2021-06-01,4200,4315,4200,4270,102406,0.016667,NaN,NaN,NaN,NaN,NaN
2021-06-01,4170,4315,4165,4215,154936,0.013221,NaN,NaN,NaN,NaN,NaN
2021-06-01,7600,7900,7540,7720,391226,0.030708,NaN,NaN,NaN,NaN,NaN


In [401]:
df['rtn20'] = df['Close'].pct_change(20)
df['rtn60'] = df['Close'].pct_change(60)
df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
df

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-01-04,3115,3115,2910,2960,442743,-0.047006,NaN,NaN,-0.048986,-0.121622,-0.128378
2021-01-05,2915,3025,2885,2955,331928,-0.001689,NaN,NaN,-0.050761,-0.118443,-0.108291
2021-01-06,2965,2990,2855,2860,299641,-0.032149,NaN,NaN,-0.010490,-0.094406,-0.104895
2021-01-07,2890,2930,2830,2885,254451,0.008741,NaN,NaN,-0.038128,-0.095321,-0.105719
2021-01-08,2890,2930,2755,2800,325189,-0.029463,NaN,NaN,0.012500,-0.044643,-0.067857
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-09,3155,3180,3000,3070,279022,-0.039124,-0.135211,-0.229611,NaN,NaN,NaN
2022-05-10,2975,3065,2860,3065,200285,-0.001629,-0.136620,-0.219108,NaN,NaN,NaN
2022-05-11,3000,3050,2950,2990,72211,-0.024470,-0.143266,-0.213158,NaN,NaN,NaN


In [408]:
df[df.index == '2021-06-01']

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-06-01,3551,3551,3416,3476,146410,0.005787,-0.079693,0.334357,0.056387,0.047756,0.066456


In [409]:
df = fdr.DataReader('094970','2021')
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,2900,2940,2850,2880,190342,-0.006897
2021-01-05,2880,2925,2855,2885,177356,0.001736
2021-01-06,2890,2960,2840,2955,427182,0.024263
2021-01-07,2960,3025,2930,2995,532612,0.013536
2021-01-08,2995,3035,2950,2990,361336,-0.001669
...,...,...,...,...,...,...
2022-05-09,3955,3965,3820,3840,135724,-0.037594
2022-05-10,3660,3920,3640,3920,213895,0.020833
2022-05-11,3920,3955,3870,3910,56952,-0.002551


In [410]:
df['rtn20'] = df['Close'].pct_change(20)
df['rtn60'] = df['Close'].pct_change(60)
df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
df

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-01-04,2900,2940,2850,2880,190342,-0.006897,NaN,NaN,0.243056,0.250000,0.003472
2021-01-05,2880,2925,2855,2885,177356,0.001736,NaN,NaN,0.230503,0.285962,0.025997
2021-01-06,2890,2960,2840,2955,427182,0.024263,NaN,NaN,0.219966,0.172589,0.018613
2021-01-07,2960,3025,2930,2995,532612,0.013536,NaN,NaN,0.265442,0.126878,-0.008347
2021-01-08,2995,3035,2950,2990,361336,-0.001669,NaN,NaN,0.299331,0.117057,-0.018395
...,...,...,...,...,...,...,...,...,...,...,...
2022-05-09,3955,3965,3820,3840,135724,-0.037594,-0.055351,-0.057669,NaN,NaN,NaN
2022-05-10,3660,3920,3640,3920,213895,0.020833,-0.034483,-0.022444,NaN,NaN,NaN
2022-05-11,3920,3955,3870,3910,56952,-0.002551,-0.034568,-0.024938,NaN,NaN,NaN


In [411]:
df[df.index == '2021-09-01']

,Open,High,Low,Close,Volume,Change,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,
2021-09-01,3805,4065,3805,4020,240303,0.052356,-0.110619,0.005,-0.121891,-0.113184,-0.206468


In [279]:
def rtn20_60(df):
    df['rtn20'] = df['Close'].pct_change(20)
    df['rtn60'] = df['Close'].pct_change(60)
    df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
    df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
    df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
    return df

In [ ]:
low20_marcap_data_codes[0]

In [351]:
concodes

['000995',
 '002785',
 '096870',
 '189860',
 '36328K',
 '065420',
 '072130',
 '900340',
 '011300',
 '238120',
 '115610',
 '032580']

In [280]:
# 데이터프레임을 만들어서 인덱스만 뽑아내고 콘캣 
conmet_rtn20 =[]
for i in concodes:
    df = fdr.DataReader(i,'2021')
    df['code'] = df['Close'].apply(lambda x:i)    
    conmet_rtn20.append(rtn20_60(df))

In [281]:
conmet_rtn20

[              Open    High     Low   Close  Volume    Change    code  \
 Date                                                                   
 2021-04-01  103000  104500  101500  103500     503  0.004854  000995   
 2021-04-02  105500  105500  102000  102500     574 -0.009662  000995   
 2021-04-05  104000  109000  103000  108000    1065  0.053659  000995   
 2021-04-06  110000  114500  108500  114500    1746  0.060185  000995   
 2021-04-07  114000  114000  111500  113500     676 -0.008734  000995   
 2021-04-08  111500  113500  110500  110500     702 -0.026432  000995   
 2021-04-09  112500  112500  110000  110000     484 -0.004525  000995   
 2021-04-12  110500  114000  110000  114000     341  0.036364  000995   
 2021-04-13  115500  115500  114000  115000    1139  0.008772  000995   
 2021-04-14  117000  117000  115000  116000     532  0.008696  000995   
 2021-04-15  117500  117500  115000  115500     537 -0.004310  000995   
 2021-04-16  115000  115000  108500  113000     727

In [282]:
rtnconmetdf = pd.concat(conmet_rtn20)
rtnconmetdf

,Open,High,Low,Close,Volume,Change,code,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,,
2021-04-01,103000,104500,101500,103500,503,0.004854,000995,0.068111,-0.100000,0.198068,0.149758,0.169082
2021-04-02,105500,105500,102000,102500,574,-0.009662,000995,0.075551,-0.096916,0.180488,0.200000,0.156098
2021-04-05,104000,109000,103000,108000,1065,0.053659,000995,0.155080,-0.129032,0.125000,0.157407,0.092593
2021-04-06,110000,114500,108500,114500,1746,0.060185,000995,0.254107,-0.053719,0.061135,0.091703,0.013100
2021-04-07,114000,114000,111500,113500,676,-0.008734,000995,0.233696,-0.061983,0.061674,0.114537,0.074890
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-19,1905,1930,1845,1865,891078,-0.020997,032580,0.113433,0.180380,NaN,NaN,NaN
2022-04-20,1885,1900,1860,1890,678852,0.013405,032580,0.148936,0.181250,NaN,NaN,NaN
2022-04-21,1945,1995,1885,1905,2959402,0.007937,032580,0.144144,0.179567,NaN,NaN,NaN


In [283]:
df202106_conmetcode

['002785', '096870', '189860']

In [284]:
temp_lis = []
temp_df = rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[0]].loc['2021-06-01']

In [285]:
temp_lis.append(pd.DataFrame(temp_df).T)
pd.concat(temp_lis)

,Open,High,Low,Close,Volume,Change,code,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
2021-06-01,7080,7250,6900,7240,26288,0.034286,002785,0.080597,0.458207,-0.129834,-0.006906,0.052486


In [286]:
rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[2]].loc['2021-06-01']

KeyError: '2021-06-01'

In [287]:
temp_lis = []

In [288]:
for i in range(len(df202103_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202103_conmetcode[i]].loc['2021-03-02']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [289]:
temp_lis

[]

In [290]:
for i in range(len(df202106_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202106_conmetcode[i]].loc['2021-06-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [291]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [292]:
for i in range(len(df202109_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202109_conmetcode[i]].loc['2021-09-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [293]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [294]:
for i in range(len(df202112_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202112_conmetcode[i]].loc['2021-12-01']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [295]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ]

In [296]:
for i in range(len(df202203_conmetcode)):
    try:
        temp_df = rtnconmetdf[rtnconmetdf['code'] == df202203_conmetcode[i]].loc['2022-03-02']
        temp_lis.append(pd.DataFrame(temp_df).T)
    except:
        pass

In [297]:
temp_lis

[            Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  7080  7250  6900  7240  26288  0.034286  002785  0.080597   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.458207 -0.129834 -0.006906  0.052486  ,
             Open  High   Low Close Volume    Change    code     rtn20  \
 2021-06-01  5180  5180  5010  5120  98457 -0.011583  096870  0.088204   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2021-06-01  0.253366  0.083984  0.251953  0.070312  ,
             Open  High   Low Close  Volume    Change    code     rtn20  \
 2022-03-02  1690  1710  1640  1705  835145  0.008876  032580  0.249084   
 
                rtn60 rtn60fore rtn40fore rtn20fore  
 2022-03-02  0.121711       NaN -0.002933  0.067449  ]

# 결과 

In [298]:
final_df = pd.concat(temp_lis)
final_df

,Open,High,Low,Close,Volume,Change,code,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
2021-06-01,7080,7250,6900,7240,26288,0.034286,002785,0.080597,0.458207,-0.129834,-0.006906,0.052486
2021-06-01,5180,5180,5010,5120,98457,-0.011583,096870,0.088204,0.253366,0.083984,0.251953,0.070312
2022-03-02,1690,1710,1640,1705,835145,0.008876,032580,0.249084,0.121711,NaN,-0.002933,0.067449


In [299]:
# 분기별 조건들을 모두 충족하는 기업들에 투자했을때 60일 수익률 평균 
final_df['rtn60fore'].mean()*100

-2.2924939571823204

In [300]:
# 분기별 조건들을 모두 충족하는 기업들에 투자했을때 40일 수익률 평균 
final_df['rtn40fore'].mean()*100

8.070483211076187

In [301]:
# 분기별 조건들을 모두 충족하는 기업들에 투자했을때 20일 수익률 평균 
final_df['rtn20fore'].mean()*100

6.341578939907001

In [ ]:
# 더쉽게하는거?

In [348]:
def rtn20_60(df):
    df['rtn20'] = df['Close'].pct_change(20)
    df['rtn60'] = df['Close'].pct_change(60)
    df['rtn60fore'] = (df.Close.shift(-60) - df.Close) / df.Close 
    df['rtn40fore'] = (df.Close.shift(-40) - df.Close) / df.Close 
    df['rtn20fore'] = (df.Close.shift(-20) - df.Close) / df.Close 
    df = df[df['rtn20'] > 0.05]
    return df

In [330]:
# 시총데이터에서 분기날짜에 수익률 5퍼이상인종목의 하위 20퍼센트를 뽑음
tmplisq = rtn20_60(marcapdf.loc['2021-03-02']).sort_values(ascending=True,by='Marcap').reset_index().loc[:int(len(rtn20_60(marcapdf.loc['2021-03-02']))*0.2)]['Code'].tolist()
# tmpdf

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rtn20'] = df['Close'].pct_change(20)
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rtn60'] = df['Close'].pct_change(60)
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [331]:
tmplisq

['179720',
 '001529',
 '000325',
 '267810',
 '009275',
 '215050',
 '002787',
 '032685',
 '001527',
 '329020',
 '329050',
 '002785',
 '001525',
 '014915',
 '000227',
 '005965',
 '245450',
 '001065',
 '016385',
 '021045',
 '001067',
 '270210',
 '000545',
 '090355',
 '093510',
 '311060',
 '347140',
 '004545',
 '011155',
 '270020',
 '310870',
 '005745',
 '26490K',
 '08537M',
 '086460',
 '001755',
 '121060',
 '202960',
 '014825',
 '004985',
 '002995',
 '285770',
 '332710',
 '366330',
 '199290',
 '00806K',
 '341160',
 '307070',
 '005725',
 '004565',
 '351340',
 '355150',
 '000995',
 '317030',
 '354230',
 '333430',
 '322190',
 '008355',
 '299670',
 '004415',
 '348840',
 '001745',
 '367340',
 '217320',
 '004255',
 '210120',
 '009415',
 '002025',
 '323350',
 '084440',
 '145995',
 '236030',
 '120780',
 '287410',
 '000725',
 '008500',
 '327970',
 '112190',
 '000075',
 '281310',
 '278990',
 '007815',
 '178600',
 '160600',
 '224760',
 '221800',
 '238170',
 '025890',
 '004835',
 '153360',
 '163430',

In [332]:
df_fs

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
000020,2020/12,687.0,70.0,80.0,4338.0,911.0,3427.0,9.04,7.09,278.0,19.25,11999.0,1.64,20
000020,2021/03,718.0,51.0,55.0,4373.0,961.0,3412.0,10.08,7.86,195.0,12.56,11941.0,1.19,20
000020,2021/06,758.0,72.0,70.0,4471.0,960.0,3511.0,9.61,7.73,224.0,13.99,12272.0,1.28,20
000020,2021/09,691.0,37.0,8.0,4479.0,947.0,3531.0,5.98,4.89,16.0,25.02,12302.0,1.45,20
000020,2021/12,763.0,65.0,62.0,4478.0,877.0,3601.0,5.27,4.44,204.0,22.82,12534.0,1.16,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
950220,2020/12,NaN,-108.0,-107.0,956.0,114.0,842.0,NaN,NaN,-130.0,NaN,1018.0,0.00,950220
950220,2021/03,NaN,-108.0,-108.0,2040.0,162.0,1879.0,NaN,NaN,-125.0,NaN,1909.0,5.89,950220
950220,2021/06,NaN,-137.0,-137.0,1919.0,155.0,1764.0,-29.88,-27.65,-139.0,NaN,1792.0,5.94,950220


In [333]:
# df[df.index == '2021-09-01']
# 2021년 1분기 
# 시총데이터에서 분기날짜에 수익률 5퍼이상인종목의 하위 20퍼센트를 뽑은데이터를 재무제표에서 찾음 
low20lis2 = []
for i in tmplisq:
    low20lis2.append(df_fs[df_fs.index == i])

In [337]:
pd.concat(low20lis2)[pd.concat(low20lis2)['level_1'] == '2021/03']

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code
mod_code,,,,,,,,,,,,,,
001529,2021/03,1560.0,-32.0,42.0,9911.0,2157.0,7754.0,-2.12,-2.23,18.0,NaN,3937.0,0.35,1529
000325,2021/03,2031.0,62.0,61.0,10776.0,5006.0,5770.0,3.68,2.77,145.0,12.78,35615.0,0.36,325
009275,2021/03,1878.0,42.0,44.0,4321.0,2421.0,1900.0,-1.33,-1.04,58.0,NaN,2707.0,0.55,9275
002787,2021/03,796.0,89.0,93.0,2890.0,1609.0,1281.0,24.38,9.04,63.0,18.28,874.0,3.95,2787
032685,2021/03,171.0,33.0,31.0,640.0,179.0,461.0,NaN,NaN,82.0,21.72,1120.0,1.60,32685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
087600,2021/03,94.0,-13.0,-10.0,874.0,80.0,794.0,1.51,1.39,-128.0,50.83,12469.0,0.60,87600
073010,2021/03,95.0,4.0,6.0,475.0,229.0,246.0,4.85,2.47,16.0,57.90,681.0,2.73,73010
002820,2021/03,1071.0,18.0,27.0,5103.0,3034.0,2069.0,-6.32,-2.94,221.0,NaN,17433.0,0.27,2820


In [350]:
# 조건을 충족하는 코드뽑쓰 
conmet(pd.concat(low20lis2)[pd.concat(low20lis2)['level_1'] == '2021/03'])
# pd.concat(low20lis2)[pd.concat(low20lis2)['level_1'] == '2021/03']

,level_1,매출액,영업이익,당기순이익,자산총계,부채총계,자본총계,ROE(%),ROA(%),EPS(원),PER(배),BPS(원),PBR(배),code,전분기대비매출성장률,영업이익률
mod_code,,,,,,,,,,,,,,,,
000995,2021/03,2437.0,606.0,478.0,11962.0,3678.0,8285.0,20.98,13.55,1074.0,15.78,19062.0,2.95,995,100.0,24.866639


In [ ]:
# 코드치고 수익률 확인 

In [347]:
ab =rtn20_60(marcapdf.loc['2021-03-02'])
ab[ab['Code'] == '000995']

C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rtn20'] = df['Close'].pct_change(20)
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rtn60'] = df['Close'].pct_change(60)
C:\Users\bitcamp\AppData\Local\Temp/ipykernel_6704/3880747745.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

,Code,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,...,Amount,Marcap,Stocks,MarketId,Rank,rtn20,rtn60,rtn60fore,rtn40fore,rtn20fore
Date,,,,,,,,,,,,,,,,,,,,,
2021-03-02,000995,DB하이텍1우,KOSPI,NaN,99600.0,1,3100.0,3.21,97700.0,100000.0,...,39837400.0,1.118667e+10,112316,STK,2421,7.197531,19.326531,-0.978916,-0.979167,-0.979418
